In [ ]:
# %% [markdown]
# Combined Agentic Search & Prometheus Monitoring
# %%
import os
import json
import re
import time
import requests
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from kubernetes import client, config
from kubernetes.client.exceptions import ApiException
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# LangGraph imports
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from langchain_core.tools import tool
from io import StringIO
import sys

# Setup - Replace with your actual API key
os.environ["NRP_API_KEY"] = "NRP-API-key-here"  # Replace with your actual key
config.load_incluster_config()
v1 = client.CoreV1Api()
apps_v1 = client.AppsV1Api()
batch_v1 = client.BatchV1Api()
networking_v1 = client.NetworkingV1Api()

# OpenAI client
from openai import OpenAI
client = OpenAI(
    api_key=os.environ.get("NRP_API_KEY"),
    base_url="https://llm.nrp-nautilus.io/"
)

# %% [markdown]
# Documentation Knowledge Base
# %%
class DocumentationKnowledgeBase:
    def __init__(self):
        self.documents = []
        self.embeddings = None
        self.metadata = []
        self.api_key = os.environ.get("NRP_API_KEY", "NRP-API-key-here")
        self.base_url = "https://llm.nrp-nautilus.io/"
        self.embedding_endpoint = f"{self.base_url}/v1/embeddings"
        self.rerank_endpoint = f"{self.base_url}/v1/rerank"
        
        # Create a robust session with retries
        self.session = requests.Session()
        retry_strategy = Retry(
            total=3,
            backoff_factor=1,
            status_forcelist=[429, 500, 502, 503, 504],
            allowed_methods=["GET", "POST"]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self.session.mount("http://", adapter)
        self.session.mount("https://", adapter)
        self.session.headers.update({
            "User-Agent": "NRP-Documentation-Crawler/1.0",
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        })
        
    def crawl_documentation(self, base_url, max_depth=1, delay=2, timeout=30):
        """Crawl the NRP.ai documentation with specified depth"""
        visited_urls = set()
        pages_to_crawl = [(base_url, 0)]
        failed_urls = []
        
        while pages_to_crawl:
            url, depth = pages_to_crawl.pop(0)
            
            if depth > max_depth or url in visited_urls:
                continue
                
            visited_urls.add(url)
            time.sleep(delay)  # Delay between requests
            
            try:
                print(f"Crawling: {url} (depth: {depth})")
                response = self.session.get(url, timeout=timeout)
                
                if response.status_code != 200:
                    print(f"Failed to fetch {url}: Status {response.status_code}")
                    failed_urls.append(url)
                    continue
                    
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # Extract page content
                title = soup.find('title').get_text() if soup.find('title') else "No Title"
                
                # Try to find the main content area
                content_div = soup.find('div', class_='documentation') or \
                              soup.find('main') or \
                              soup.find('article') or \
                              soup.find('div', class_='content') or \
                              soup
                
                content = content_div.get_text(strip=True)
                
                # Skip pages with very little content
                if len(content) < 100:
                    print(f"Skipping {url} - insufficient content")
                    continue
                
                # Store document with metadata
                self.documents.append({
                    'text': content,
                    'url': url,
                    'title': title
                })
                
                # Find all links
                for link in soup.find_all('a', href=True):
                    href = link['href']
                    full_url = urljoin(url, href)
                    
                    # Only follow links within the documentation
                    if urlparse(full_url).netloc == urlparse(base_url).netloc:
                        pages_to_crawl.append((full_url, depth + 1))
                        
            except requests.exceptions.RequestException as e:
                print(f"Error crawling {url}: {e}")
                failed_urls.append(url)
                continue
                
        print(f"Crawled {len(self.documents)} pages")
        if failed_urls:
            print(f"Failed to crawl {len(failed_urls)} pages:")
            for url in failed_urls:
                print(f"  - {url}")
                
    def get_embeddings(self, texts, batch_size=10):
        """Get embeddings from the NRP API with batching"""
        all_embeddings = []
        
        # Process texts in batches to avoid overwhelming the API
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            data = {
                "model": "embed-mistral",
                "input": batch
            }
            
            try:
                response = self.session.post(self.embedding_endpoint, json=data, timeout=30)
                if response.status_code == 200:
                    result = response.json()
                    all_embeddings.extend([item['embedding'] for item in result['data']])
                    print(f"Processed batch {i//batch_size + 1}/{(len(texts)-1)//batch_size + 1}")
                else:
                    print(f"Error getting embeddings: {response.status_code} - {response.text}")
                    # Add zero embeddings as fallback
                    all_embeddings.extend([[0.0] * 768] * len(batch))
            except Exception as e:
                print(f"Exception when getting embeddings: {e}")
                # Add zero embeddings as fallback
                all_embeddings.extend([[0.0] * 768] * len(batch))
                
            # Add delay between batches
            time.sleep(1)
            
        return all_embeddings
    
    def rerank_results(self, query, documents, top_k=5):
        """Rerank search results using the NRP API"""
        # Prepare documents for reranking
        docs_for_rerank = [{"text": doc['text']} for doc in documents]
        
        data = {
            "model": "gemma3",
            "query": query,
            "documents": docs_for_rerank,
            "top_n": top_k
        }
        
        try:
            response = self.session.post(self.rerank_endpoint, json=data, timeout=30)
            if response.status_code == 200:
                result = response.json()
                # Get indices of top results
                top_indices = [item['index'] for item in result['results']]
                return [documents[i] for i in top_indices]
            else:
                print(f"Error reranking results: {response.status_code} - {response.text}")
                return documents[:top_k]  # Fallback to original order
        except Exception as e:
            print(f"Exception when reranking: {e}")
            return documents[:top_k]  # Fallback to original order
    
    def search(self, query, top_k=5, use_reranking=True):
        """Search the knowledge base"""
        if self.embeddings is None:
            print("Knowledge base not loaded. Please load it first.")
            return []
            
        # Get query embedding
        query_embedding = self.get_embeddings([query])
        if query_embedding is None:
            print("Failed to generate query embedding")
            return []
            
        query_embedding = np.array(query_embedding[0]).reshape(1, -1)
        
        # Calculate similarity
        similarities = cosine_similarity(query_embedding, self.embeddings).flatten()
        
        # Get top results
        top_indices = similarities.argsort()[-top_k:][::-1]
        
        # Prepare results
        results = []
        for idx in top_indices:
            results.append({
                'text': self.documents[idx]['text'],
                'url': self.metadata[idx]['url'],
                'title': self.metadata[idx]['title'],
                'score': float(similarities[idx])
            })
        
        # Apply reranking if requested
        if use_reranking and len(results) > 0:
            results = self.rerank_results(query, results, top_k)
            
        return results
    
    def build_knowledge_base(self):
        """Build the knowledge base with embeddings"""
        if not self.documents:
            print("No documents to process. Please crawl the documentation first.")
            return
            
        # Get embeddings for all documents
        print("Generating embeddings...")
        texts = [doc['text'] for doc in self.documents]
        embeddings = self.get_embeddings(texts)
        
        if embeddings is None:
            print("Failed to generate embeddings")
            return
            
        self.embeddings = np.array(embeddings)
        self.metadata = [{
            'url': doc['url'],
            'title': doc['title']
        } for doc in self.documents]
        
        print(f"Knowledge base built with {len(self.documents)} documents")
    
    def save_knowledge_base(self, filepath):
        """Save the knowledge base to disk"""
        if self.embeddings is None:
            print("Knowledge base not built. Nothing to save.")
            return
            
        data = {
            'documents': self.documents,
            'embeddings': self.embeddings.tolist(),
            'metadata': self.metadata
        }
        
        with open(filepath, 'w') as f:
            json.dump(data, f)
            
        print(f"Knowledge base saved to {filepath}")
    
    def load_knowledge_base(self, filepath):
        """Load the knowledge base from disk"""
        with open(filepath, 'r') as f:
            data = json.load(f)
            
        self.documents = data['documents']
        self.embeddings = np.array(data['embeddings'])
        self.metadata = data['metadata']
        
        print(f"Knowledge base loaded from {filepath} with {len(self.documents)} documents")

# Initialize Documentation Knowledge Base
doc_kb = DocumentationKnowledgeBase()
kb_file = "nrp_expert_docs_kb.json"
if os.path.exists(kb_file):
    doc_kb.load_knowledge_base(kb_file)
else:
    print(f"Knowledge base file {kb_file} not found. Building it now with crawl depth 1...")
    doc_kb.crawl_documentation("https://nrp.ai/documentation/", max_depth=1)
    doc_kb.build_knowledge_base()
    doc_kb.save_knowledge_base(kb_file)

# %% [markdown]
# Prometheus Monitoring Functions
# %%
def describe_pods(namespace="gsoc"):
    """Describe pods and print only fields useful for Prometheus metric queries."""
    try:
        pods = v1.list_namespaced_pod(namespace=namespace) if namespace else v1.list_pod_for_all_namespaces()
        rows = []
        for pod in pods.items:
            pod_name = pod.metadata.name
            ns = pod.metadata.namespace
            pod_ip = pod.status.pod_ip
            node = pod.spec.node_name
            container_names = [c.name for c in pod.spec.containers]
            container = ", ".join(container_names)
            rows.append([pod_name, ns, pod_ip, node, container])
        headers = ["Pod", "Namespace", "Pod IP", "Node", "Container"]
        return tabulate(rows, headers=headers, tablefmt="fancy_grid")
    except ApiException as e:
        return f"❌ Error fetching pods: {e}"

def namespace_gpu_utilization(prom_url="https://prometheus.nrp-nautilus.io", threshold=0):
    """Display average GPU utilization per namespace using PromQL."""
    query = 'avg by (namespace) (DCGM_FI_DEV_GPU_UTIL)'
    url = f"{prom_url}/api/v1/query"
    try:
        response = requests.get(url, params={"query": query}, timeout=10)
        response.raise_for_status()
        data = response.json()
        if data.get("status") != "success":
            return "❌ Prometheus query failed."
        results = data["data"]["result"]
        if not results:
            return "✅ Query successful, but no GPU usage data returned."
        rows = []
        for r in results:
            ns = r["metric"].get("namespace", "unknown")
            util = float(r["value"][1])
            if util >= threshold:
                status = (
                    "🟢 Low" if util < 40 else
                    "🟡 Moderate" if util < 70 else
                    "🔴 High"
                )
                rows.append([ns, f"{util:.2f}%", status])
        headers = ["Namespace", "Avg GPU Utilization", "Status"]
        return tabulate(rows, headers=headers, tablefmt="fancy_grid")
    except Exception as e:
        return f"❌ Error querying Prometheus: {e}"

def fetch_dcgm_gpu_util_data(prom_url="https://prometheus.nrp-nautilus.io"):
    """Fetch rich GPU utilization data from Prometheus using DCGM_FI_DEV_GPU_UTIL."""
    query = 'DCGM_FI_DEV_GPU_UTIL'
    url = f"{prom_url}/api/v1/query"
    try:
        response = requests.get(url, params={"query": query}, timeout=10)
        response.raise_for_status()
        data = response.json()
        if data.get("status") != "success":
            return []
        results = data["data"]["result"]
        if not results:
            return []
        enriched = []
        for r in results:
            m = r["metric"]
            val = float(r["value"][1])
            enriched.append({
                "hostname": m.get("Hostname", "unknown"),
                "ip_port": m.get("instance", "unknown"),
                "gpu_id": m.get("gpu", "N/A"),
                "device": m.get("device", "N/A"),
                "uuid": m.get("UUID", "N/A"),
                "model": m.get("modelName", "unknown"),
                "namespace": m.get("namespace", "N/A"),
                "pod": m.get("pod", "N/A"),
                "utilization": val
            })
        return enriched
    except Exception as e:
        return []

def analyze_dcgm_gpu_data(data):
    """Analyze DCGM GPU data with statistics and top utilization."""
    if not data:
        return "No data to analyze."
    total = len(data)
    avg_util = sum(d["utilization"] for d in data) / total
    maxed = [d for d in data if d["utilization"] >= 99.0]
    idle = [d for d in data if d["utilization"] < 1.0]
    available = [d for d in data if d["utilization"] < 100.0]
    unique_hosts = set(d["hostname"] for d in data)
    unique_models = set(d["model"] for d in data)
    
    result = f"""🔍 Total GPUs: {total}
📊 Average Utilization: {avg_util:.2f}%
🔴 Fully Utilized GPUs (>=99%): {len(maxed)}
🟢 Idle GPUs (<1%): {len(idle)}
💻 Unique Host Machines: {len(unique_hosts)}
🧠 Unique GPU Models: {len(unique_models)}
🧮 GPUs Available (<100%): {len(available)}\n"""
    
    result += "📈 Top 10 GPUs by Utilization:\n"
    top = sorted(data, key=lambda x: x["utilization"], reverse=True)[:10]
    rows = [[d["hostname"], d["gpu_id"], d["model"], f"{d['utilization']:.2f}%", d["namespace"], d["pod"]] for d in top]
    result += tabulate(rows, headers=["Host", "GPU", "Model", "Utilization", "Namespace", "Pod"], tablefmt="github")
    return result

# %% [markdown]
# Tool Definitions
# %%
# Helper to capture and truncate printed output
def capture_stdout_truncated(func, max_length=2000, *args, **kwargs):
    """Capture stdout and truncate if too long to prevent LLM loops"""
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    try:
        result = func(*args, **kwargs)
    finally:
        sys.stdout = old_stdout
    
    output = mystdout.getvalue()
    if len(output) > max_length:
        output = output[:max_length] + f"\n\n... [Output truncated - showing first {max_length} characters]"
    return output

# Documentation search tool
@tool
def search_documentation(query: str) -> str:
    """Search the NRP.ai documentation for relevant information."""
    if doc_kb.embeddings is None:
        return "❌ Knowledge base not loaded. Cannot search documentation."
    
    results = doc_kb.search(query, top_k=3)
    if not results:
        return "❌ No relevant documentation found."
    
    output = []
    for i, result in enumerate(results, 1):
        output.append(f"Result {i}:")
        output.append(f"Title: {result['title']}")
        output.append(f"URL: {result['url']}")
        output.append(f"Content: {result['text'][:200]}...")
        output.append("")  # Empty line
    
    return "\n".join(output)

# GPU monitoring tools
@tool
def describe_pods_tool(namespace: str = "gsoc") -> str:
    """Describe pods in a given Kubernetes namespace. Defaults to 'gsoc'."""
    return capture_stdout_truncated(describe_pods, 1500, namespace=namespace)

@tool
def namespace_gpu_util_tool(threshold: float = 0.0) -> str:
    """Get average GPU utilization per namespace with optional threshold filter."""
    return capture_stdout_truncated(namespace_gpu_utilization, 1500, threshold=threshold)

@tool
def dcgm_gpu_inspect_tool(threshold: float = 0.0) -> str:
    """Inspect raw GPU usage with model name, host, pod, and utilization. Shows top 10 GPUs above threshold."""
    data = fetch_dcgm_gpu_util_data()
    if not data:
        return "⚠️ No GPU data available."
    filtered = [d for d in data if d["utilization"] >= threshold]
    if not filtered:
        return f"✅ No GPUs over {threshold}% utilization."
    top = sorted(filtered, key=lambda x: x["utilization"], reverse=True)[:10]
    rows = [
        [d["hostname"][:20], d["gpu_id"], d["model"][:25], f"{d['utilization']:.2f}%", d["namespace"], d["pod"][:20]]
        for d in top
    ]
    result = tabulate(rows, headers=["Host", "GPU", "Model", "Util%", "Namespace", "Pod"], tablefmt="grid")
    result += f"\n\nShowing top 10 of {len(filtered)} GPUs above {threshold}% threshold."
    return result

@tool
def calculate_dcgm_gpu_stats(threshold: float = 0.0) -> str:
    """Analyze GPU utilization across nodes and return statistical breakdown."""
    data = fetch_dcgm_gpu_util_data()
    if not data:
        return "⚠️ No GPU data available."
    filtered = [d for d in data if d["utilization"] >= threshold]
    total = len(filtered)
    if total == 0:
        return f"✅ No GPUs over the threshold of {threshold}% utilization."
    avg_util = sum(d["utilization"] for d in filtered) / total
    maxed = [d for d in filtered if d["utilization"] >= 99.0]
    idle = [d for d in filtered if d["utilization"] < 1.0]
    moderate = [d for d in filtered if 1.0 <= d["utilization"] < 70.0]
    available = [d for d in filtered if d["utilization"] < 100.0]
    unique_models = set(d["model"] for d in filtered)
    unique_hosts = set(d["hostname"] for d in filtered)
    return f"""📊 GPU Utilization Stats (threshold: {threshold}%):
🔍 Total GPUs: {total}
📈 Average Utilization: {avg_util:.2f}%
🔴 Fully Utilized (>=99%): {len(maxed)}
🟢 Idle (<1%): {len(idle)}
⚙️ Moderate (1-70%): {len(moderate)}
💻 Unique Hosts: {len(unique_hosts)}
🧠 Unique Models: {len(unique_models)}
🧮 Available (<100%): {len(available)}"""

# %% [markdown]
# LangGraph Agent Implementation
# %%
class NRPModel:
    def __init__(self, client):
        self.client = client
        self.tools = []
    def bind_tools(self, tools):
        self.tools = tools
        return self
    def _convert_tool_to_openai_format(self, tool):
        """Convert LangChain tool to OpenAI tool format"""
        return {
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.args_schema.model_json_schema() if tool.args_schema else {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
    def invoke(self, messages):
        # Convert messages to proper format
        formatted_messages = []
        for msg in messages:
            if hasattr(msg, 'content'):
                if msg.__class__.__name__ == "SystemMessage":
                    formatted_messages.append({"role": "system", "content": msg.content})
                elif msg.__class__.__name__ == "HumanMessage":
                    formatted_messages.append({"role": "user", "content": msg.content})
                elif msg.__class__.__name__ == "AIMessage":
                    formatted_messages.append({"role": "assistant", "content": msg.content})
                elif msg.__class__.__name__ == "ToolMessage":
                    # Truncate tool message content if too long
                    content = str(msg.content)
                    if len(content) > 2000:
                        content = content[:2000] + "\n[Content truncated...]"
                    formatted_messages.append({
                        "role": "tool", 
                        "content": content,
                        "tool_call_id": getattr(msg, 'tool_call_id', 'unknown')
                    })
            else:
                formatted_messages.append(msg)
        # Convert tools to OpenAI format
        openai_tools = None
        if self.tools:
            openai_tools = [self._convert_tool_to_openai_format(t) for t in self.tools]
        try:
            response = self.client.chat.completions.create(
                model="gemma3",
                temperature=0,
                messages=formatted_messages,
                tool_choice="auto" if openai_tools else None,
                tools=openai_tools,
            )
            choice = response.choices[0].message
            tool_calls = []
            if hasattr(choice, "tool_calls") and choice.tool_calls:
                for t in choice.tool_calls:
                    args = t.function.arguments
                    if isinstance(args, str):
                        try:
                            args = json.loads(args)
                        except json.JSONDecodeError:
                            args = {}
                    tool_calls.append({
                        "name": t.function.name,
                        "args": args,
                        "id": t.id
                    })
            return AIMessage(
                content=choice.content or "",
                tool_calls=tool_calls
            )
        except Exception as e:
            return AIMessage(content=f"Error calling model: {str(e)}")

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class Agent:
    def __init__(self, model, tools, system: str = ""):
        self.system = system
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
        self.max_iterations = 5  # Prevent infinite loops
        self.current_iteration = 0
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.raw_graph = graph
        self.graph = graph.compile()
    
    def exists_action(self, state: AgentState) -> bool:
        """Check if the last message has tool calls and we haven't exceeded max iterations"""
        if self.current_iteration >= self.max_iterations:
            print(f"⚠️ Reached max iterations ({self.max_iterations}). Stopping.")
            return False
            
        try:
            result = state["messages"][-1]
            return (hasattr(result, "tool_calls") and 
                    result.tool_calls is not None and 
                    len(result.tool_calls) > 0)
        except (IndexError, KeyError, AttributeError):
            return False
    
    def call_openai(self, state: AgentState) -> dict:
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}
    
    def take_action(self, state: AgentState) -> dict:
        self.current_iteration += 1
        tool_calls = state["messages"][-1].tool_calls
        results = []
        
        for t in tool_calls:
            tool_name = t["name"]
            tool_args = t["args"]
            print(f"🔧 Calling tool: {tool_name} with args: {tool_args}")
            
            if tool_name not in self.tools:
                result = "❌ Tool name not recognized. Available tools: " + ", ".join(self.tools.keys())
            else:
                try:
                    result = self.tools[tool_name].invoke(tool_args)
                    # Ensure result is string and truncate if needed
                    result = str(result)
                    if len(result) > 3000:
                        result = result[:3000] + "\n\n[Output truncated to prevent loops]"
                except Exception as e:
                    result = f"❌ Tool error: {str(e)}"
            
            results.append(ToolMessage(tool_call_id=t["id"], name=tool_name, content=result))
        
        print("✅ Tool(s) executed. Returning to model.")
        return {"messages": results}

# %% [markdown]
# Agent Initialization
# %%
# System prompt combining documentation search and monitoring
system_prompt = """You are a Kubernetes monitoring assistant with access to NRP.ai documentation. 
Use these tools to answer questions:
- 'search_documentation': Search NRP.ai documentation for conceptual information
- 'describe_pods_tool': View pod/container info in a namespace
- 'namespace_gpu_util_tool': View average GPU utilization per namespace  
- 'dcgm_gpu_inspect_tool': View detailed GPU metrics (top 10 results)
- 'calculate_dcgm_gpu_stats': Get statistical breakdown of GPU usage

Guidelines:
1. For conceptual questions about Kubernetes, GPU usage, or cloud computing, use 'search_documentation'
2. For current cluster status or resource utilization questions, use the monitoring tools
3. Only call each tool ONCE per question
4. Use the tool output to provide a direct answer
5. Do not repeat tool calls
6. For complex queries, break them down into multiple tool calls if needed"""

# Create agent with combined tools
model = NRPModel(client)
tools = [
    search_documentation,
    describe_pods_tool,
    namespace_gpu_util_tool,
    dcgm_gpu_inspect_tool,
    calculate_dcgm_gpu_stats
]
abot = Agent(model=model, tools=tools, system=system_prompt)

# %% [markdown]
# Query Function
# %%
def ask_agent(question):
    """Ask the agent a question and get a response"""
    abot.current_iteration = 0  # Reset counter
    messages = [HumanMessage(content=question)]
    response = abot.graph.invoke({"messages": messages})
    return response["messages"][-1].content

# %% [markdown]
# Test Cases
# %%
print("=== Test 1: Documentation Search ===")
print(ask_agent("How does GPU scheduling work in Kubernetes?"))

print("\n=== Test 2: Current GPU Utilization ===")
print(ask_agent("What's the current GPU utilization across all namespaces?"))

print("\n=== Test 3: Detailed GPU Analysis ===")
print(ask_agent("Show me detailed GPU statistics and identify any idle GPUs"))

print("\n=== Test 4: Combined Query ===")
print(ask_agent("I'm having issues with GPU scheduling. Can you check current utilization and explain best practices?"))

print("\n=== Test 5: A100 Analysis ===")
print(ask_agent("Analyze all A100 GPUs in the cluster. Show utilization, availability, and which namespaces are using them"))